In [7]:
import sys
from pathlib import Path
home = str(Path.home())
core_dir = home+'/repositories/ai-x/core'
sys.path.insert(0, core_dir)

from misc import check_output_dir

# control variables here 
chembl_version = 'C33'
n = 0 # how much you're reserving as validation dataset 
n_models = 10
#output directory, where your scripts are created
pre_modeldir = "/home/wons2/repositories/ai-DR/models/selectivity_D2D3/selectivity_models"
pre_datadir = "/home/wons2/repositories/ai-DR/models/selectivity_D2D3/selectivity_datasets"

model_dir = pre_modeldir+f'/models_{chembl_version}_{n}/'
cpu = 48
memory = 200
days = 1
hours = 0
mode = "reg"
methods = ["dnn"]
stage = "buildmodel"


####################################### CODE ###################################################
models = ['models_D2', 'models_D3', 'models__ratio_D2D3']
dict_dataset = {models[0]: 'dataset_D2_antagonist_Ki', 
                models[1]: 'dataset_D3_antagonist_Ki',
               models[2]:'dataset__ratio_D3_antagonist_Ki_D2_antagonist_Ki'}
dict_job_name = {models[0]: 'D2', 
                models[1]: 'D3', 
               models[2]:'DR'}

for method in methods:
    for model in models:
        model_dir = f'{pre_modeldir}/models_{chembl_version}_{n}'
        check_output_dir(f"{model_dir}/{model}", keep_old = False)

        for i in range(n_models):
            path = f"{model_dir}/{model}"

            filename = 'pubdata'
            val_filename = 'val'

            datadir = f"{pre_datadir}/{chembl_version}_{n}/{dict_dataset[model]}"
            dataset = f"{datadir}/{filename}"
            val_dataset = f"{datadir}/{val_filename}"
            if mode == "class":
                dataset = dataset+"_class"
                val_dataset = val_dataset+"_class"
            with open(path+f'/do_shap_{method}{i}.sh', 'w') as f:
                f.write('#!/bin/bash\n')
                f.write(f"#SBATCH --job-name=SH_{dict_job_name[model]}\n")
                f.write(f"#SBATCH --partition=norm\n")
                f.write(f"#SBATCH --time={days}-{hours}:00:00\n")
                f.write(f"#SBATCH --cpus-per-task={cpu}\n")
                f.write(f"#SBATCH --mem={memory}g\n\n")
                f.write(f"/data/Shilab/apps/anaconda3-2020.02-py3.7.6/bin/python $HOME/repositories/ai-x/core/run_deeplearning.py -s shap -m {mode} -x {method} -t 0   -r 10 -n 1 -e {i}:{i+1} -i {dataset}0\n")
                    
        # script to run one dnn model only
        with open(path+f'/all_shap_{method}.sh', 'w') as f:
            f.write('#!/bin/bash\n')
            f.write("for i in {9..0}\n")
            f.write("do\n")
            f.write("echo $i\n")
            f.write(f"sbatch do_shap_{method}$i.sh\n")
            f.write("done\n")
            # script to run one dnn model only